In [1]:
#!/usr/bin/env python3
# -*- coding: utf-8 -*-

In [2]:
# check to see if camera is available
!ls -ltrh /dev/video*

crw-rw----+ 1 root video 81, 0 Nov 10 23:59 /dev/video0


In [3]:
import cv2

In [4]:
def gstreamer_pipeline(capture_width=1280,
                       capture_height=720,
                       display_width=1280,
                       display_height=720,
                       framerate=60,
                       flip_method=0):
    """
    returns a GStreamer pipeline for capturing from the CSI camera
    Defaults to 1280x720 @ 60fps
    """
    
    return (
        'nvarguscamerasrc ! '
        'video/x-raw(memory:NVMM), '
        'width=(int)%d, height=(int)%d, '
        'format=(string)NV12, framerate=(fraction)%d/1 ! '
        'nvvidconv flip-method=%d ! '
        'video/x-raw, width=(int)%d, height=(int)%d, format=(string)BGRx ! '
        'videoconvert ! '
        'video/x-raw, format=(string)BGR ! appsink'
        % (
            capture_width,
            capture_height,
            framerate,
            flip_method,
            display_width,
            display_height,
        ))

In [5]:
def show_camera():
    print(gstreamer_pipeline(flip_method=0))
    
    # load from CSI Camera(raspi camera v2)
    cap = cv2.VideoCapture(gstreamer_pipeline(flip_method=0),
                           cv2.CAP_GSTREAMER)
    
    # check camera normally
    if cap.isOpened():
        window_handle = cv2.namedWindow('CSI Camera', cv2.WINDOW_AUTOSIZE)  # window setting
        
        # when window closed: return -1
        while cv2.getWindowProperty('CSI Camera', 0) >= 0:
            ret_val, img = cap.read()
            cv2.imshow('CSI Camera', img)
            key_code = cv2.waitKey(30) & 0xFF
            
            # Stop program with ESC key
            if key_code == 27:
                break
        
        # Freeing camera object
        cap.release()
        cv2.destroyAllWindows()
    
    else:
        print('Unable to open camera')

In [6]:
show_camera()

nvarguscamerasrc ! video/x-raw(memory:NVMM), width=(int)1280, height=(int)720, format=(string)NV12, framerate=(fraction)60/1 ! nvvidconv flip-method=0 ! video/x-raw, width=(int)1280, height=(int)720, format=(string)BGRx ! videoconvert ! video/x-raw, format=(string)BGR ! appsink
